# Практика обучения моделей

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2021/blob/master/02-pytorch/seminar.ipynb)


План семинара:

- [ ] попробовать писать логи tensorboard
- [ ] научиться сохранять/доставать чекпоинты
- [ ] разобраться с Dataset API
- [ ] написать базовый Trainer - класс с тренировочным циклом

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


from tqdm.auto import tqdm

## Логирование


Популярным вариантом для хранения логов является tensorboard: это формат хранения (protobuf, как json только бинарный) + готовый просмотрщик.




[Tensorboard](https://www.tensorflow.org/tensorboard) -- это pip-installable web-приложение.

```
tensorboard --logdirs=./some-folder/with/events-files
# зайти на http://localhost:6006
```
<img src="./img/tb.png"/>

При желании, можно написать python-код для парсинга логов и делать что-то с ними руками.

Чтобы писать логи в pytorch есть класс `torch.utils.tensorboard.SummaryWriter`

In [ ]:
writer = SummaryWriter("./check-this/")

fake_loss = 1 / np.arange(1, 100)
for global_step, point in enumerate(fake_loss):
    writer.add_scalar("lossy", point, global_step=global_step)
writer.close()

## Запускаем TensorBoard

В случае локального запуска (или на своем сервере) потребуется поставить tensorboard
```
pip install tensorboard

tensorboard --logdir=./check-this
# зайти на http://localhost:6006
```

Для работы в colab есть специальное расширение. Запустите следующие ячейки:

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./check-this

# Сохранение-загрузка тензоров и моделек

Нам часто бывает необходимо сохранить/загрузить веса модели в файл на диске. 
Распространенное название для этого -- checkpoint. 

У торчевых моделей (наследников torch.nn.Module) и оптимизаторов (наследников torch.optim.Optimizer) есть методы для получения и загрузки состояний:

`.state_dict()` возвращает словарь (или почти словарь) с весами

`.load_state_dict(some_dict)` загружает веса из словаря в модельку

Для сохранения/загрузки словарей с тензорами в файлы есть простые функции `torch.save(some_dict, path)` и `torch.load(path)`. Сравните с использованием pickle или json!

**NB: В DL термином checkpointing называют так же метод бекпропа, позволяющий экономить память ценой дополнительных вычислений (https://pytorch.org/docs/stable/checkpoint.html#torch-utils-checkpoint).**

In [ ]:
some_model = nn.Sequential(nn.Linear(10, 10))
print(some_model.state_dict())

opt = optim.Adam(some_model.parameters())
print(opt.state_dict())


torch.save({"model_stuff": some_model.state_dict(), "opt_stuff": opt.state_dict()}, "./that.is.it")

In [ ]:
torch.load("./that.is.it")

## Dataset API

Подготовка данных легко может стать бутылочным горлышком, когда на подготовку очередного батча уходит больше времени, чем на forward+backward проходы по сети.
Проблема усложняется особенностями python: чтобы использовать несколько ядер CPU для подготовки данных надо постараться.

В Pytorch работа с данными строится на двух классах из [torch.utils.data](https://pytorch.org/docs/stable/data.html): `Dataset` и `DataLoader`:

- `Dataset` отвечает за подготовку одного примера
- `DataLoader` отвечает за выбор примеров, склейку их в один батч и распараллеливание на CPU, поддерживает итерирование.


Для решения задачи обычно пишут кастомные Dataset-классы, для этого нужно написать всего две функции:
- `.__len__(self)` возвращает количество примеров в датасете;
- `.__getitem__(self, item)` возвращает item-ный по счету пример из датасета.

Задачи DataLoader достаточно сложно аккуратно реализовать и лучше использовать готовый. Он довольно гибкий, все основные моменты кастомизируются заданием функций:
```
torch.utils.data.DataLoader(
    dataset,            # собственно экземпляр класса Dataset, из которого надо доставать примеры
    batch_size=1,       # количество примеров в батче
    drop_last=False,    # нужно ли при итерировании выбрасывать неполные батчи? (такое бывает, если число примеров не делится нацело на batch_size
    shuffle=False,      # перемешивать ли примеры
    sampler=None,       # чтобы перемешивать примеры кастомно
    batch_sampler=None, # чтобы использовать кастомный отбор примеров в батч
    num_workers=0,      # на сколько процессов запараллелить подготовку данных
    collate_fn=None,    # функция, которая будет склеивать примеры в батчи
    # остальные аргументы более технические, сейчас можно не рассматривать
    pin_memory=False,   
    timeout=0, 
    worker_init_fn=None, 
    multiprocessing_context=None, 
    generator=None)
```

Напишите два датасета для работы с FashionMnist: один готовит данные как вектора, другой как картинки


**NB: FashionMNIST возвращает картинки в формате PIL.Image.Image, чтобы сделать из него понятный np.array, просто вызовите np.array(PIL_IMAGE)**

In [ ]:
from torchvision.datasets import FashionMNIST


class VectorSet:
    def __init__(self, train=True):
        self.data = FashionMNIST("./tmp", train=train, download=True)
    
    def __len__(self):
        <your code>
    
    def __getitem__(self, item):
        # сделайте вектор с float32 числами
        <your code>
        return dict(
            sample=...,
            label=...,
        )

vs = VectorSet()
print(vs[0])
        
class ImageSet:
    def __init__(self, train=True):
        self.data = FashionMNIST("./tmp", train=train, download=True)
    
    def __len__(self):
        <your code>
    
    def __getitem__(self, item):
        # сделайте одноканальную картинку [1, 28, 28] с float32
        <your code>
        return dict(
            sample=...,
            label=...,
        )
    
ms = ImageSet()
print(ms[0])

In [ ]:
# проверьте итерирование, именно его мы используем в train-loop'е
vl = DataLoader(vs, batch_size=4)
for batch in vl:
    for k, v in batch.items():
        print(k, v.shape)
    raise

## Замечания по Dataset/Dataloader

1. Dataset может возвращать что угодно (туплы, словари, whatever) с отдельными числами или массивами (numpy, torch.tensor).
Удобно возвращать словари с читабельными ключами, тогда будет проще разделять логику по компонентам.

2. Имеет смысл поглядеть в [стандартный collate_fn](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/_utils/collate.py#L42): он умеет клеить в батчи и конвертировать в тензора самые разнообразные данные. Это может работать во многих случаях, но неожиданно падать в других. В частности, не сможет поклеить примеры разной длины.


# Classy Trainer

Пишем класс для тренировки и логгирования.

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, train_dataset, val_dataset, batch_size=128):
        self.model = model
        self.optimizer = optimizer
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset

        self.batch_size = batch_size

        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)
        
        self.global_step = 0
        self.writer = SummaryWriter("./tmp/")

    def save_checkpoint(self, path):
        torch.save(self.model.state_dict(), path)

    def train(self, num_epochs):
        model = self.model
        optimizer = self.optimizer
        
        train_loader = DataLoader(self.train_dataset, shuffle=True, pin_memory=True, batch_size=self.batch_size)
        val_loader = DataLoader(self.val_dataset, shuffle=False, pin_memory=True, batch_size=self.batch_size)
        best_loss = float('inf')
        
        for epoch in range(num_epochs):
            model.train()
            for batch in tqdm(train_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                for k, v in details.items():
                    self.writer.add_scalar(k, v, global_step=self.global_step)
                self.global_step += 1
            
            model.eval()
            
            val_losses = []
            for batch in tqdm(val_loader):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                loss, details = model.compute_all(batch)
                val_losses.append(loss.item())
                
            val_loss = np.mean(val_losses)        
            if val_loss < best_loss:
                self.save_checkpoint("./best_checkpoint.pth")
                best_loss = val_loss

                



In [ ]:
vs = VectorSet()
print(vs[0])

In [ ]:
class VeryModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.inner = nn.Sequential(nn.Linear(784, 100), nn.ReLU(), nn.Linear(100, 10))
    
    def forward(self, x):
        return self.inner(x)
    
    def compute_all(self, batch):  # удобно сделать функцию, в которой вычисляется лосс по пришедшему батчу
        x = batch['sample']
        y = batch['label']
        logits = self.inner(x)
        
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(axis=1) == y).float().mean().cpu().numpy()
        metrics = dict(acc=acc)
        return loss, metrics

# проверяйте работоспособность сразу
model = VeryModel()
opt = optim.SGD(model.parameters(), lr=1e-2)
trainset = VectorSet(train=True)
valset = VectorSet(train=False)

trainer = Trainer(model, opt, trainset, valset, batch_size=128)

In [ ]:
trainer.train(10)

Просмотрите логи локально или через tensorboard-ext в зависимости от того, как вы запустили тетрадку